In [1]:
# !pip install opencv
# !pip install tensorflow
# !pip install pandas
# !pip install pillow
# !pip install pydicom
# !pip install pandas-profiling

In [ ]:
# Hola Mundo!

# Como fue el proyecto al final?
# [] Bien
# [] Mal
# [] Que buena pregunta